In [ ]:
import numpy as np
import cv2
import time
import os
from tensorflow.keras.models import load_model


In [ ]:
!pip install ujson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
import ujson as json
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in sorted(os.listdir(folder)):  # 파일명에 따라 정렬 ##
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [ ]:
def main():

    input_folder_path = "/content/drive/MyDrive/데이터캡스톤/images/input_yolo"
    output_folder_path = "/content/drive/MyDrive/데이터캡스톤/images/output_images"
    all_images = load_images_from_folder(input_folder_path)
    print('Total Images: ',len(all_images))
    HEIGHT = 32
    WIDTH = 32

    net = cv2.dnn.readNet("/content/drive/MyDrive/데이터캡스톤/yolov4-custom_last.weights", "/content/drive/MyDrive/데이터캡스톤/yolov4-custom.cfg")

    classes = []
    with open("/content/drive/MyDrive/데이터캡스톤/signs_classes_yolo.txt", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    #get last layers names
    layer_names = net.getLayerNames()
    output_layers = []
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    check_time = True
    confidence_threshold = 0.5
    font = cv2.FONT_HERSHEY_SIMPLEX


    detection_confidence = 0.5
    cap = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX

    #accuracy를 위해 추가된 부분
    records = map(json.loads, open('/content/drive/MyDrive/데이터캡스톤/final.ndjson'))
    df = pd.DataFrame.from_records(records)
    df['external_id'] = df['data_row'].apply(lambda x: x['external_id']) #사진명대로 오름차순 정렬
    sorted_df = df.sort_values(by='external_id', ascending=True)


    classification_model = load_model('/content/drive/MyDrive/데이터캡스톤/traffic_sign_classifier.h5') #load mask detection model
    classes_classification = []
    with open("/content/drive/MyDrive/데이터캡스톤/signs_classes_cnn.txt", "r") as f:
        classes_classification = [line.strip() for line in f.readlines()]


        classification_results = []

        input_files = sorted(os.listdir(input_folder_path)) ##
        all_images = [cv2.imread(os.path.join(input_folder_path, file)) for file in input_files]
        for i,img in enumerate(all_images):
            file_name=input_files[i]
            #get image shape
            height, width, channels = img.shape
            #print(img.shape)

            image_result = {
            'index': i,
            'filename':file_name,
            'classification': [],
            'accuracy': 0
            }
            image_result['filename']=file_name
        #추가/

        # Detecting objects (YOLO)
            blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

        # Showing informations on the screen (YOLO)
            class_ids = []
            confidences = []
            boxes = []
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > confidence_threshold:
                        # Object detected
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        # Rectangle coordinates
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)
            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)


            for i in range(len(boxes)):
                if i in indexes:
                    x, y, w, h = boxes[i]
                    label = str(classes[class_ids[i]]) + "=" + str(round(confidences[i]*100, 2)) + "%"
                    img = cv2.rectangle(img, (x, y), (x + w, y + h), (255,0,0), 2)
                    crop_img = img[y:y+h, x:x+w]
                    if len(crop_img) >0:
                        crop_img = cv2.resize(crop_img, (WIDTH, HEIGHT))
                        crop_img =  crop_img.reshape(-1, WIDTH,HEIGHT,3)
                        prediction = np.argmax(classification_model.predict(crop_img))
                        print(prediction)
                        predicted_label = classes_classification[prediction]
                        label = str(classes_classification[prediction])
                        img = cv2.putText(img, label, (x, y), font, 0.5, (255,0,0), 2)
                        image_result['classification'].append(label)

        # 분류 결과 저장
            classification_results.append(image_result)
            output_image_path = os.path.join("/content/drive/MyDrive/데이터캡스톤/images/output_images", file_name)
            cv2.imwrite(output_image_path, img)

    # 이미지 순서대로 분류 및 정확도 출력
        classification_results= sorted(classification_results, key=lambda x: x['filename'])
        correct_count=0
        total_count=0

        actual_label=[]
        for result in classification_results:
            file_name = result['filename']
            matched_row = df[df['external_id'] == file_name]
            if len(matched_row) == 0:
                print(f"No matching data row for image {file_name}.")
            else:
                actual_label = matched_row['projects'].values[0]['clhdb6c7q1v2307457ier1b4k']['labels'][0]['annotations']['classifications'][0]['checklist_answers'][0]['name']

            predicted_labels = result['classification']

            if any(label in actual_label for label in predicted_labels):
                correct_count += 1
            total_count += 1
            accuracy = 0 if total_count == 0 else correct_count / total_count * 100
            result['accuracy'] = accuracy


            print("Image:", result['filename'])
            for label in result['classification']:
                print("Detected Object:", label)
            print("Accuracy: {:.2f}%".format(result['accuracy']))


main()

Total Images:  505
1/1 [==============================] - 0s 98ms/step
12
1/1 [==============================] - 0s 18ms/step
18
1/1 [==============================] - 0s 19ms/step
25
1/1 [==============================] - 0s 19ms/step
0
1/1 [==============================] - 0s 19ms/step
25
1/1 [==============================] - 0s 18ms/step
4
1/1 [==============================] - 0s 17ms/step
9
1/1 [==============================] - 0s 18ms/step
21
1/1 [==============================] - 0s 19ms/step
2
1/1 [==============================] - 0s 20ms/step
40
1/1 [==============================] - 0s 20ms/step
2
1/1 [==============================] - 0s 18ms/step
9
1/1 [==============================] - 0s 19ms/step
40
1/1 [==============================] - 0s 17ms/step
40
1/1 [==============================] - 0s 18ms/step
4
1/1 [==============================] - 0s 17ms/step
42
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 17ms/step
23